## smart platform testing

## lib

In [112]:
import os
import pandas as pd

### data preprocessing

In [117]:
def get_csv_files(folder_path):
    csv_files=[]
    for files in os.listdir(folder_path):
        if files.endswith('.csv'):
            csv_files.append(os.path.join(folder_path,files))
    return csv_files

csv_files= get_csv_files('../amazon-products-dataset/versions/2')

In [118]:
# store the data in dat frame
dataframes= []
for path in csv_files:
    df = pd.read_csv(path)
    dataframes.append(df)
    
combined_df= pd.concat(dataframes,ignore_index=True)


### basic info

In [119]:
combined_df.head()

,name,main_category,sub_category,image,link,ratings,no_of_ratings,discount_price,actual_price,Unnamed: 0
0,Puma Women's Glam WNS Sport Sandal,women's shoes,Fashion Sandals,https://m.media-amazon.com/images/I/51cVFnTFCS...,https://www.amazon.in/Puma-Womens-Glam-Lotus-S...,4.5,2,"₹1,829","₹2,999",NaN
1,Luxyfeel Fashion Sandal for Girls and Women | ...,women's shoes,Fashion Sandals,https://m.media-amazon.com/images/I/51IBSgyvAK...,https://www.amazon.in/Luxyfeel-Fashion-Sandal-...,3.8,44,₹549,"₹1,499",NaN
2,STRASSE PARIS Amazing Design Women's & Girls T...,women's shoes,Fashion Sandals,https://m.media-amazon.com/images/I/616TZINkHa...,https://www.amazon.in/STRASSE-PARIS-Transparen...,4.5,2,₹636,"₹1,899",NaN
3,Mochi Women White Wedge Heel Slip-on UK/5 EU/3...,women's shoes,Fashion Sandals,https://m.media-amazon.com/images/I/51nDP-iEI6...,https://www.amazon.in/Mochi-Women-White-Wedge-...,5.0,1,"₹1,194","₹1,990",NaN
4,Catwalk Women's Fashion UK EU US 3882,women's shoes,Fashion Sandals,https://m.media-amazon.com/images/I/712tmgiNOn...,https://www.amazon.in/Catwalk-Womens-Fashion-S...,3.7,114,₹899,"₹2,195",NaN


In [120]:
combined_df = combined_df.drop(combined_df.columns[-1], axis=1)
combined_df.drop(['name','image','link'],axis=1,inplace=True)

In [121]:
combined_df.shape

(1103170, 6)

In [125]:
combined_df=combined_df.dropna()

In [126]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 681360 entries, 0 to 1103169
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   main_category   681360 non-null  object
 1   sub_category    681360 non-null  object
 2   ratings         681360 non-null  object
 3   no_of_ratings   681360 non-null  object
 4   discount_price  681360 non-null  object
 5   actual_price    681360 non-null  object
dtypes: object(6)
memory usage: 36.4+ MB


In [127]:
combined_df.columns = (combined_df.columns.str.strip().str.lower().str.replace(" ","_"))

In [128]:
combined_df= combined_df[
    pd.to_numeric(combined_df['ratings'],errors= 'coerce').between(0,5)
]
combined_df['ratings'] = combined_df['ratings'].astype(float)

In [129]:
combined_df['no_of_ratings']= (
    combined_df['no_of_ratings'].astype(str)
    .str.replace(',','')
    .where(lambda x:x.str.fullmatch(r'\d+'))
    .astype(float)
)

In [130]:
price_pattern = r'^₹\d{1,3}(,\d{3})*$'
cols = ['discount_price','actual_price']

masked = (
    combined_df[cols]
    .astype(str)
    .apply(lambda x:x.str.match(price_pattern))
    .all(axis=1)
)

combined_df = combined_df[masked]

combined_df[cols] = (
    combined_df[cols]
    .replace({'₹':'',',':''},regex=True)
    .astype(float)
)

In [131]:
combined_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 650204 entries, 0 to 1102940
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   main_category   650204 non-null  object 
 1   sub_category    650204 non-null  object 
 2   ratings         650204 non-null  float64
 3   no_of_ratings   650108 non-null  float64
 4   discount_price  650204 non-null  float64
 5   actual_price    650204 non-null  float64
dtypes: float64(4), object(2)
memory usage: 34.7+ MB


In [132]:
combined_df['main_category'].unique()

array(["women's shoes", 'car & motorbike', 'appliances',
       'grocery & gourmet foods', "kids' fashion", 'bags & luggage',
       'toys & baby products', "women's clothing", 'tv, audio & cameras',
       "men's shoes", 'sports & fitness', 'beauty & health',
       'accessories', "men's clothing", 'home & kitchen', 'music',
       'pet supplies', 'stores', 'industrial supplies',
       'home, kitchen, pets'], dtype=object)